In [1]:
from lib import geoSearch
import json

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
GS = geoSearch.GeoSearch("elasticsearch-geo",9200)
from shapely.geometry import mapping, shape

In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-geo', 'port': 9200}])
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"search_calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [4]:
text = "Desvo por riesgo en estructura de puente Desde 13 de marzo Tristn Narvaja entre Cerro Largo y La Paz Lneas urbanas y suburbanas: Av. Uruguay &gt; Tristn Narvaja &gt; Cerro Largo &gt; Minas &gt; Miguelete &gt; Fernndez Crespo a su ruta https://t.co/PFMpX6pFiF"
analyzed_text = analyzedText(text)

In [5]:
matches = GS.complete_search(text)
for nombre, match in matches:
    if match and len(match['objects']) == 1:
        shape_1 = shape(match['objects'][0]['geometry'])
        print ('✔️ MATCH: {}\t/\t{}'.format(match['score'],nombre))
        #print(json.dumps(mapping(shape_1.representative_point())))
    elif match and len(match['objects']) > 1:
        shape_1 = shape(match['objects'][0]['geometry'])
        shape_2 = shape(match['objects'][1]['geometry'])
        print ('✔️ MATCH: {}\t/\t{}'.format(match['score'],nombre))
        #print mapping(shape_1.intersection(shape_2).representative_point())
    print('\n')

✔️ MATCH: 26468.8580688	/	cerro-largo|tristan-narvaja


✔️ MATCH: 4429.61165816	/	av-uruguay|minas


✔️ MATCH: 3913.46159123	/	la-paz|miguelete


✔️ MATCH: 409.272890772	/	cap-felipe-ruiz-puente|cap-pedro-de-mesa-y-castro


✔️ MATCH: 131.860659111	/	cerro-norte


✔️ MATCH: 130.471876542	/	cerro-de-montevideo


✔️ MATCH: 128.068488627	/	arroyo-del-miguelete


✔️ MATCH: 124.270906693	/	estacin-la-paz


✔️ MATCH: 115.134601357	/	el-cerro-light


✔️ MATCH: 112.408799369	/	cuchilla-de-miguelete


✔️ MATCH: 111.651795514	/	cerro-de-las-espinas


✔️ MATCH: 96.8777202954	/	fortaleza-del-cerro


✔️ MATCH: 78.621272172	/	playa-del-cerro


✔️ MATCH: 70.7764050498	/	villa-del-cerro


✔️ MATCH: 65.1965145689	/	ceadu---(centro-de-estudios,-anlisis-y-comunicacin-del-uruguay)


✔️ MATCH: 58.1342290803	/	sociedad-de-arquitectos-del-uruguay-(sau)


✔️ MATCH: 49.0394576557	/	unicef-uruguay


✔️ MATCH: 49.0394576557	/	cempre-uruguay


✔️ MATCH: 48.7236387509	/	edificio-asociacion-de-empleados-bancarios-de

In [6]:
def boosting_match_bool_search2(text, size=500, boost=2, negative_boost=0.5):
    """
    Elasticsearch query. I'm trying to boost results that doesn't come from negative_types or must_not terms.
    """
    return es.search(index='montevideo', body=
    {
        "from": 0, "size": size,
        "query": {

                    "bool": {
                        "should": [
                            { "match": { "nombre":            text }},
                            { "match": { "nombre.variant_1":  text }},
                            { "match": { "nombre.variant_2":  text }},
                            { "match": { "aliases":           text }},
                            { "match": { "aliases.variant_1": text }},
                            { "match": { "aliases.variant_2": text }}
                          ]
                    }
                },
            
        
         "highlight": {
                "fields": {
                    "*": {}
                },
             "number_of_fragments": 10
            }
    })['hits']['hits']

In [ ]:
import re
for res in boosting_match_bool_search2(text):
    obj = GS.get_result_object(res)
    print(obj['name'],obj['score'],obj['highlights'],obj['geo_type'])

In [ ]:
print(obj)

In [9]:
import pandas as pd
df = pd.read_pickle("./df_second_method.pkl")

In [10]:
df

,text,name,score,time_ms,new_name,new_score
0,Este Jueves 12 de Marzo - 22 hs. Se presenta M...,montevideo,386.291,1221.995,3-pla-al-sur-de-conciliacion|del-cid,701.200214
1,"Resumen, fecha 3\n\nMontevideo City Torque 1...",francisco-rodrigo|jose-nicolas-oxilia,2099.856,1093.838,francisco-rodrigo|jose-nicolas-oxilia,7035.536122
2,Maldita ciudad (Montevideo),montevideo,212.998,64.929,baha-de-montevideo,156.884735
3,Ex vivero municipal cuenca del Caneln chico re...,ro-santa-luca,219.898,548.124,marcelino-diaz-y-garcia|santa-lucia,2757.223855
4,@MiguelAsqueta acabo de viajar a desde bs as ...,sda-q|sda-x,792.430,101.876,sda-a-a|sda-q,1418.107909
5,"Este sbado es el Montevideo Rock, un espectcul...",chile|republica-argentina,943.078,203.510,republica-argentina|chile,1969.617318
6,Sorteamos 3 entradas dobles para #MontevideoR...,3-de-15-mts|7-de-12-mts,201.389,509.494,,0.000000
7,"Para @AlfredoAsti, @CosseCarolina, @rotambor y...",sda-q|sda-x,1584.860,182.934,sda-a-d|sda-q,3369.926773
8,Sorteamos 3 entradas dobles para #MontevideoR...,3-de-15-mts|7-de-12-mts,201.389,443.735,,0.000000
9,@jmartinezjorge @MSPUruguay @montevideoIM @chd...,3|2,254.080,1005.794,,0.000000


In [15]:
for index, row in df.iterrows():
    print(row.text.replace('\n',' '))
    print(row.name, row.score)
    matches = GS.complete_search(row.text)
    for nombre, match in matches[:4]:
        if match and len(match['objects']) == 1:
            shape_1 = shape(match['objects'][0]['geometry'])
            print ('✔️ MATCH: {}\t/\t{}'.format(match['score'],nombre))
            #print(json.dumps(mapping(shape_1.representative_point())))
        elif match and len(match['objects']) > 1:
            shape_1 = shape(match['objects'][0]['geometry'])
            shape_2 = shape(match['objects'][1]['geometry'])
            print ('✔️ MATCH: {}\t/\t{}'.format(match['score'],nombre))
            #print mapping(shape_1.intersection(shape_2).representative_point())
    print('\n')

Este Jueves 12 de Marzo - 22 hs. Se presenta MONTEVIDEO JAZZ' en Museo del Vino.   El cuarteto Montevideo Jazz propone un repertorio con obras de reconocidos autores del jazz, otras de autores nacionales tradas al https://t.co/VMq26q0QY6
(0, 386.291)
✔️ MATCH: 199.319311814	/	bolsa-de-valores-(montevideo)
✔️ MATCH: 165.913428592	/	3-pla-al-sur-de-conciliacion|del-cid
✔️ MATCH: 165.913428592	/	5-pla-al-sur-de-conciliacion|del-cid
✔️ MATCH: 160.599361015	/	departamento-de-montevideo


 Resumen, fecha 3  Montevideo City Torque  1  1  Cerro   Jos Tancredi (20') y Nicols Ramos (Gol en contra - 41')    Rodrigo Izquierdo (20')   3/3   Se anot el primer gol en contra del torneo https://t.co/SIhemsubRV
(1, 2099.856)
✔️ MATCH: 115.834553953	/	cerro-de-montevideo
✔️ MATCH: 101.724600843	/	fortaleza-del-cerro
✔️ MATCH: 95.2738548979	/	el-cerro-light
✔️ MATCH: 70.3395144836	/	smart-montevideo


Maldita ciudad (Montevideo)
(2, 212.998)
✔️ MATCH: 70.3395144836	/	montevideo-shopping
✔️ MATCH: 70.33951

✔️ MATCH: 73.3845895079	/	plaza-casavalle-un-lugar-para-todos-plaza
✔️ MATCH: 69.2686459492	/	obelisco-a-los-constituyentes-de-1830-(montevideo)
✔️ MATCH: 44.0068312024	/	fundacion-nios-con-alas
✔️ MATCH: 35.2942334594	/	los-ceibos|el-ombu


Este sbado es el Montevideo Rock, un espectculo dnde vienen personas a verlo de todas partes de latinoamerica, principalmente argentina y chile. Se estima que asistan 50.000 personas. No habra que suspenderlo? Es un peligro! @chdica hac algo til por favor!!
(20, 943.078)
✔️ MATCH: 731.600456253	/	republica-argentina|chile
✔️ MATCH: 318.92453272	/	rbla-republica-argentina|santiago-de-chile
✔️ MATCH: 147.394853433	/	liceo-no-28-(montevideo)
✔️ MATCH: 120.474771132	/	liceo-no-7-joaquin-suarez-(montevideo)


Yuri Debenedetti  Empleado de la IMM Coautor de Difamacion junto a Daniel Pieiro, contra la Policia.  Militante a Sueldo pago por la IMM ( Daniel Martinez Di Candia y cia)  Cada vez se wntiende mas el "no perder lo conquistado" Delincuentes FA. Inf

✔️ MATCH: 2379.21263385	/	francisco-acua-de-figueroa|lima
✔️ MATCH: 123.346370973	/	calderon-de-la-barca|francisco-de-los-santos
✔️ MATCH: 57.8168556798	/	plaza-de-la-armada
✔️ MATCH: 53.8416656317	/	figueroa-plazuela


 Sorteamos 3 entradas dobles para #MontevideoRock!    Dale MG y RT   Seguinos  # Mencion a un amigo/a con #MontevideoRock.   El 12/3 anunciamos a los ganadores. https://t.co/ltY0pVrmxx
(37, 201.389)
✔️ MATCH: 73.3845895079	/	plaza-casavalle-un-lugar-para-todos-plaza
✔️ MATCH: 69.2686459492	/	obelisco-a-los-constituyentes-de-1830-(montevideo)
✔️ MATCH: 44.0068312024	/	fundacion-nios-con-alas
✔️ MATCH: 35.2942334594	/	los-ceibos|el-ombu


El @ALA_Congreso se llevar a cabo del 24 al 28 de noviembre en #Montevideo, #Uruguay.  Si quieres participar como ponente ests a tiempo de registrarte: https://t.co/BvqoCD5lRi https://t.co/stgz3v6dyv
(38, 366.563)
✔️ MATCH: 1034.28201047	/	18-de-noviembre|el-remanso
✔️ MATCH: 138.098915123	/	sociedad-de-arquitectos-del-uruguay-(sau)
✔️ M

✔️ MATCH: 2350.67796997	/	bolsa-de-valores-(montevideo)
✔️ MATCH: 364.022586282	/	baha-de-montevideo
✔️ MATCH: 354.521493111	/	departamento-de-montevideo
✔️ MATCH: 342.839918335	/	puerto-de-montevideo


@Descerebrado17 @jmartinezjorge @MSPUruguay @montevideoIM Todos hablan del Montevideo Rock,ayer estuvo Karol G,Domingo y Lunes Batris Boys,en el estadio Maroon 5,Ricky Martn y no se sabe de nada
(55, 407.53)
✔️ MATCH: 147.394853433	/	liceo-no-28-(montevideo)
✔️ MATCH: 120.474771132	/	liceo-no-7-joaquin-suarez-(montevideo)
✔️ MATCH: 108.381695532	/	escuela-no-17-brasil-(montevideo)
✔️ MATCH: 93.4656921377	/	bolsa-de-valores-(montevideo)


Gran gestion la de @Dmartinez_uy seguida de @chdica , gastan mal, no usan lo que invierten y despues lo sacan. Asi cuidan la plata de los Montevideanos. Para cambiar esto en mayo votamos a @lauraraffo para Intendenta! Si, Podemos!
(56, 468.13)
✔️ MATCH: 5737.08437927	/	rio-de-la-plata|mal-abrigo
✔️ MATCH: 1333.3915445	/	calderon-de-la-barca|francisco-de

✔️ MATCH: 73.3845895079	/	plaza-casavalle-un-lugar-para-todos-plaza
✔️ MATCH: 69.2686459492	/	obelisco-a-los-constituyentes-de-1830-(montevideo)
✔️ MATCH: 44.0068312024	/	fundacion-nios-con-alas
✔️ MATCH: 35.2942334594	/	los-ceibos|el-ombu


Como les encanta cortar la rambla por cualquier cosa, ahora Montevideo Rock, tambin lo hacen por maratones, etc. Me parece que una de las dos vas principales de la ciudad no puede cortarse continuamente.
(72, 212.998)
✔️ MATCH: 193.305775946	/	arroyo-de-la-tortuga
✔️ MATCH: 165.537998794	/	islote-de-la-luz
✔️ MATCH: 147.394853433	/	liceo-no-28-(montevideo)
✔️ MATCH: 120.474771132	/	liceo-no-7-joaquin-suarez-(montevideo)


Fernando Daniel Martnez Estrada, miembro del Ejrcito mexicano y medallista en los Juegos Panamericanos Lima 2019, fue denunciado por su ex novia, Edna Quintanilla, de haberla golpeado y amenazado de muerte https://t.co/DT9nlx3oWD https://t.co/52kdDHnQJ8
(73, 494.712)
✔️ MATCH: 1006.73541686	/	lima|av-daniel-fernandez-crespo
✔️ MAT

...

✔️ MATCH: 275.693035461	/	municipio-d
✔️ MATCH: 192.882416613	/	calle-de-la-via|el-ombu
✔️ MATCH: 123.694227084	/	banco-de-la-familia
✔️ MATCH: 44.0068312024	/	fundacion-nios-con-alas


Toque de campana en la Bolsa de Valores de Montevideo por la igualdad de gnero #GeneracinIgualdad Aqu empec mi carrera profesional y nunca haba visto tantas mujeres en la Bolsa de Valores https://t.co/6clyXiFW77
(1458, 286.073)
✔️ MATCH: 2350.67796997	/	bolsa-de-valores-(montevideo)
✔️ MATCH: 364.022586282	/	baha-de-montevideo
✔️ MATCH: 354.521493111	/	departamento-de-montevideo
✔️ MATCH: 342.839918335	/	puerto-de-montevideo


Esta plaza es un justo homenaje a Luisa. Sabemos que no pudimos decirle dnde est Nebio, llegaremos a la verdad que era la mayor justicia que ella anhelaba.  Que sea smbolo de paz y convivencia, que recuerde su lucha, y su mirada ilumine nuestro camino, an en nuestra deuda. https://t.co/zPD4Uv1b46
(1459, 600.91)
✔️ MATCH: 1489.98134256	/	cno-de-la-justicia|cno-a-la-cuchilla-pereira

✔️ MATCH: 1006.73541686	/	lima|av-daniel-fernandez-crespo
✔️ MATCH: 217.285447841	/	islote-de-la-luz
✔️ MATCH: 130.499682419	/	obelisco-a-los-constituyentes-de-1830-(montevideo)
✔️ MATCH: 103.396328401	/	plaza-de-la-armada


@imtransito @ElOjoChurrinche 2 semanas cartel tirado, esquina peligrosa. Enrique Muoz y Sarmiento. #Peligro https://t.co/wnHhWCsVej
(1475, 624.021)
✔️ MATCH: 149.154719258	/	av-sarmiento|dr-enrique-muoz
✔️ MATCH: 25.2898703183	/	edificio-altos-de-sarmiento
✔️ MATCH: 20.6683406508	/	plazoleta-enrique-ibsen
✔️ MATCH: 18.2795413519	/	monumento-a-jose-enrique-rodo


Movilizacin sbado 13/03 de 10 a 13 hs   MUNICIPIO A  Parque Belln - Paso Molino Paso de la Arena - Tomkinson y Batlle Berres https://t.co/mZFSsroJ4u
(1476, 3921.56)
✔️ MATCH: 35765.9548709	/	cno-paso-de-la-arena|av-luis-batlle-berres
✔️ MATCH: 18424.8714381	/	sbre-de-paso-de-10-mts|av-luis-batlle-berres
✔️ MATCH: 4766.74605354	/	luis-de-la-pea|av-luis-batlle-berres
✔️ MATCH: 2556.28637533	/	sbre-de-paso-de-

✔️ MATCH: 35765.9548709	/	cno-paso-de-la-arena|av-luis-batlle-berres
✔️ MATCH: 18424.8714381	/	sbre-de-paso-de-10-mts|av-luis-batlle-berres
✔️ MATCH: 4766.74605354	/	luis-de-la-pea|av-luis-batlle-berres
✔️ MATCH: 2556.28637533	/	sbre-de-paso-de-4-mts|cno-de-la-capilla


Movilizacin sbado 13/03 de 10 a 13 hs   MUNICIPIO A  Parque Belln - Paso Molino Paso de la Arena - Tomkinson y Batlle Berres https://t.co/mZFSsroJ4u
(1497, 3921.56)
✔️ MATCH: 35765.9548709	/	cno-paso-de-la-arena|av-luis-batlle-berres
✔️ MATCH: 18424.8714381	/	sbre-de-paso-de-10-mts|av-luis-batlle-berres
✔️ MATCH: 4766.74605354	/	luis-de-la-pea|av-luis-batlle-berres
✔️ MATCH: 2556.28637533	/	sbre-de-paso-de-4-mts|cno-de-la-capilla


Pero el comportamiento social en las ciudades latinoamericanas de estos das ha sido increble: Lima y Quito desabastecian los supermercados, Buenos Aires y Montevideo tranquilos, Santiago con sus propios problemas. 2/4
(1498, 2522.201)
✔️ MATCH: 2409.24302261	/	buenos-aires|santiago-de-liniers

✔️ MATCH: 43.1437260903	/	cerro-de-montevideo
✔️ MATCH: 38.3066575635	/	dr.-washington-beltran-plazoleta
✔️ MATCH: 23.767921146	/	continental-(montevideo)
✔️ MATCH: 23.767921146	/	after-montevideo


Peafiles, buenos das.   Hoy solo tengo una pregunta.   Dnde est Marcelo Ebrard?  Ebrard ha realizado 16 viajes internacionales: Marrakech, Nueva York, Montevideo, Roma, Los ngeles, Washington, Berln, Osaka, Beijing, Lima, San Salvador y Texas.  Ex gobierno federal
(1517, 940.848)
✔️ MATCH: 648.257112088	/	republica-del-salvador|washington-beltran
✔️ MATCH: 124.205626451	/	los-angeles-(montevideo)
✔️ MATCH: 82.8469335653	/	obelisco-a-los-constituyentes-de-1830-(montevideo)
✔️ MATCH: 70.3395144836	/	esplendor-montevideo


CONFIRMADO   LOS PRIMEROS 4 CASOS DE CORONAVIRUS EN URUGUAY. SON 2 EN MONTEVIDEO Y 2 EN EL INTERIOR. EN MINUTOS CONFERENCIA DE PRENSA.
(1518, 352.64)
✔️ MATCH: 87.2523802938	/	obelisco-a-los-constituyentes-de-1830-(montevideo)
✔️ MATCH: 79.9324436084	/	cerro-de-montevideo
✔️

✔️ MATCH: 162.195488167	/	cno-el-sembrador|sda-de-paso
✔️ MATCH: 162.195488167	/	sda-de-paso|cno-el-polvorin
✔️ MATCH: 124.180286038	/	cno-paso-de-la-espaola|cno-el-aguila
✔️ MATCH: 116.024156053	/	1er-pla-a-la-corona|psje-el-hornero


Me informan x la cucaracha q hay dos casos de coronavirus en Montevideo y dos en salto
(1537, 396.215)
✔️ MATCH: 470.785206617	/	salto|nuestra-seora-de-la-encina
✔️ MATCH: 47.1356658492	/	baha-de-montevideo
✔️ MATCH: 47.0407061808	/	departamento-de-montevideo
✔️ MATCH: 46.0631795507	/	puerto-de-montevideo


#DERRUMBAMIENTOS  #Artigas #Rivera #Salto  #Tacuaremb #Paysand #CerroLargo #RoNegro #Durazno #TreintayTres #Soriano #Florida #Flores #Rocha #Lavalleja #Colonia #SanJos #Maldonado #Canelones #Montevideo #Uruguay https://t.co/miLnja0gob
(1538, 1918.465)
✔️ MATCH: 2647.81394631	/	durazno|salto
✔️ MATCH: 1126.88741705	/	canelones|florida
✔️ MATCH: 1083.31235111	/	florida|soriano
✔️ MATCH: 1005.80809791	/	colonia|florida


